# 3. Работа с Big Data

In [1]:
import os
os.environ['PYSPARK_PYTHON'] = 'python'
os.environ['HADOOP_USER_NAME'] = 'root'  # Обход проверки пользователя

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder \
    .appName("Test") \
    .master("local[*]") \
    .config("spark.driver.host", "localhost") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

## 3.1. Spark SQL (каталог, метаданные)

Каталог (Catalog) — это абстракция в Spark SQL, которая предоставляет API для:
- Управления базами данных (databases)
- Работы с таблицами (tables)
- Доступа к представлениям (views)
- Хранения метаданных (схем, типов данных, партиций)

Встроенный каталог vs Hive Metastore
| Особенность | Встроенный каталог | Hive Metastore |
|- |- |- |
| Где хранятся метаданные? | В памяти (исчезают после завершения сессии) | Во внешней БД (MySQL, PostgreSQL) |
| Когда использовать? | Для временных таблиц | Для постоянных таблиц в продакшене |
| Доступ из других приложений | Нет| Да (через Hive, Impala и др.) |

Работа с каталогом через SparkSession

Показать все базы данных
`spark.catalog.listDatabases()`  
Показать все таблицы в текущей БД
`spark.catalog.listTables()`  
Показать все колонки таблицы
`spark.catalog.listColumns(df_Titanic)`

Создание и управление базами данных
Создать новую БД
`spark.sql("CREATE DATABASE IF NOT EXISTS my_db")`  
Переключиться на БД
`spark.catalog.setCurrentDatabase("my_db")`  
Удалить БД (осторожно!)
`spark.sql("DROP DATABASE IF EXISTS my_db CASCADE")`

Временные vs постоянные таблицы

| Тип | Временная таблица | Постоянная таблица |
|- |- |- |
| Видимость | Только в текущей сессии Spark | Доступна всем сессиям |
| Хранение метаданных | В памяти | В Hive Metastore |
| Хранение данных | Зависит от источника| Сохраняется на HDFS/S3 |

Временная таблица (исчезнет после завершения сессии)
`df.createOrReplaceTempView("temp_titanic")`  
Постоянная таблица (сохранится в каталоге Hive)
`df.write.saveAsTable("perm_titanic")`

***VIEW*** — это виртуальная таблица (запрос не выполняется, пока не вызвана).
***TABLE*** — материализованные данные.

```
# Временное представление
spark.sql("""
    CREATE OR REPLACE TEMP VIEW adult_passengers AS
    SELECT * FROM titanic WHERE Age >= 18
""")

# Постоянное представление (требует Hive)
spark.sql("""
    CREATE OR REPLACE VIEW perm_adult_passengers AS
    SELECT * FROM titanic WHERE Age >= 18
""")
```

Интеграция с Hive Metastore. Чтобы Spark мог работать с Hive Metastore, добавьте в spark-defaults.conf
```
spark.sql.catalogImplementation=hive
spark.hadoop.hive.metastore.uris=thrift://metastore-host:9083
```

```
# Чтение таблицы из Hive
hive_df = spark.sql("SELECT * FROM hive_db.hive_table")

# Запись данных в Hive
df.write.saveAsTable("hive_db.new_table")
```

## 3.2. Structured Streaming (основы)

Structured Streaming — это масштабируемая и отказоустойчивая система потоковой обработки данных, построенная на движке Spark SQL.

Принцип работы:
- Бесконечный DataFrame: Потоковые данные представляются как "бесконечно растущая" таблица
- Микропакетная обработка: Данные обрабатываются небольшими порциями (микропакетами)
- Exactly-once семантика: Гарантируется однократная обработка каждого события

**Источники**(Input) - Kafka, Файлы (CSV, Parquet), Сокеты (тестирование) и **приемники**(Output) - Консоль (debug), Файлы, Kafka, Базы данных (JDBC)


Создание потокового приложения. Чтение данных из сокета (для тестирования)

```
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder \
    .appName("StructuredStreaming") \
    .master("local[*]") \
    .getOrCreate()

# Создаем потоковый DataFrame
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

# Простая трансформация - подсчет длины строк
lengths = lines.select(length(col("value")).alias("length"))

# Вывод в консоль
query = lengths \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()
```
Как запустить:
- Откройте терминал и запустите Netcat: `nc -lk 9999`
- Введите несколько строк текста
- В консоли Spark появятся результаты обработки

#### Обработка потоковых данных из файлов

Мониторинг папки с новыми файлами
```
schema = StructType([
    StructField("timestamp", TimestampType(), True),
    StructField("product", StringType(), True),
    StructField("price", FloatType(), True)
])

stream_df = spark \
    .readStream \
    .schema(schema) \
    .option("maxFilesPerTrigger", 1) \  # Обрабатывать по 1 файлу за раз
    .csv("path/to/input_folder")

# Агрегация по продуктам
product_counts = stream_df.groupBy("product").count()

query = product_counts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()
```

**maxFilesPerTrigger** - контроль скорости обработки  
**latestFirst** - обрабатывать сначала новые файлы  
**cleanSource** - удалять обработанные файлы  

#### Работа с временными окнами (Window Operations)

Агрегация по временным окнам

```
from pyspark.sql.functions import window

windowed_counts = stream_df \
    .withWatermark("timestamp", "10 minutes") \  # Водяной знак
    .groupBy(
        window("timestamp", "5 minutes"),  # Окно 5 минут
        "product"
    ) \
    .agg(avg("price").alias("avg_price"))

query = windowed_counts \
    .writeStream \
    .outputMode("update") \
    .format("console") \
    .option("truncate", "false") \
    .start()
```

Водяной знак (Watermark): Позволяет обрабатывать задержавшиеся данные

Типы окон:
- Tumbling (фиксированные без перекрытия)
- Sliding (перекрывающиеся)
- Session (по активности)

#### Интеграция с Kafka 

Чтение данных из Kafka
```
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "host1:port1,host2:port2") \
    .option("subscribe", "topic1") \
    .load()

# Парсинг JSON-сообщений
from pyspark.sql.functions import from_json

json_schema = StructType([...])  # Определите схему JSON
parsed_df = df.select(
    from_json(col("value").cast("string"), json_schema).alias("data")
).select("data.*")
```

Запись в Kafka
```
query = processed_df \
    .selectExpr("to_json(struct(*)) AS value") \
    .writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("topic", "output_topic") \
    .option("checkpointLocation", "/path/to/checkpoint") \
    .start()
```

Управление состоянием и чекпоинты. Настройка чекпоинтов.
```
query = df.writeStream \
    .outputMode("append") \
    .format("parquet") \
    .option("path", "output_path") \
    .option("checkpointLocation", "checkpoint_dir") \  # Обязательно!
    .start()
```

Восстановление после сбоев  
Точный контроль над смещениями (offsets)  
Управление состоянием агрегаций

## 3.3. Оптимизация запросов (Explain, Tungsten)

## 3.4. Работа с GraphFrames

## 3.5. Интеграция с Python (Pandas, NumPy)

In [3]:
spark.stop()